In [1]:
#!pip install selenium

In [2]:
'''!pip install requests
!pip install beautifulsoup4
!pip install pandas'''

'!pip install requests\n!pip install beautifulsoup4\n!pip install pandas'

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
import os
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

In [ ]:
URL_TEMPLATE = "https://www.tokopedia.com/gameridsurabaya/product?page={}"

CLASS_KARTU_PRODUK = "Ui5-B4CDAk4Cv-cjLm4o0g== XeGJAOdlJaxl4+UD3zEJLg=="
CLASS_NAMA_PRODUK = "+tnoqZhn89+NHUA43BpiJg=="
CLASS_HARGA_PRODUK = "urMOIDHH7I0Iy1Dv2oFaNw== HJhoi0tEIlowsgSNDNWVXg=="

In [ ]:
def parse_local_files():
    semua_data = []
    
    #loop cari file halaman1.html sampai halaman9.html
    for i in range(6, 10):
        filename = f"halaman{i}.html"
        if not os.path.exists(filename):
            print(f"File {filename} tidak ditemukan. Lewati.")
            continue
        print(f"Membaca file {filename}...")
        
        with open(filename, "r", encoding="utf-8") as f:
            html_content = f.read()
            
        soup = BeautifulSoup(html_content, 'html.parser')
        
        #cari produk dgn Regex agar lebih fleksibel
        kartu_produk = soup.find_all('a', class_=re.compile(r"Ui5")) #cari yg ada tulisan 'Ui5'

        print(f"--> Ditemukan {len(kartu_produk)} calon produk.")
        
        for kartu in kartu_produk:
            try:
                nama_el = kartu.find('span', class_=CLASS_NAMA_PRODUK)
                harga_el = kartu.find('div', class_=CLASS_HARGA_PRODUK)
                
                if nama_el and harga_el:
                    nama = nama_el.text.strip()
                    harga_raw = harga_el.text.strip()
                    
                    harga_int = int("".join(filter(str.isdigit, harga_raw)))
                    
                    semua_data.append({
                        'Nama_Produk': nama,
                        'Harga': harga_int
                    })
            except:
                continue

    return semua_data

In [ ]:
data_final = parse_local_files()

if data_final:
    df = pd.DataFrame(data_final)
    df = df.drop_duplicates(subset=['Nama_Produk'])
    
    output_file = 'laptop_tokopedia_offline2.csv'
    df.to_csv(output_file, index=False)
    
    print("\n================ SUKSES ================")
    print(f"Data berhasil disimpan ke {output_file}")
    print(f"Total Data Unik: {len(df)}")
    print(df.head())
else:
    print("\nGagal. Coba cek lagi NAMA CLASS di file html Anda.")

Membaca file halaman6.html...
--> Ditemukan 80 calon produk.
Membaca file halaman7.html...
--> Ditemukan 80 calon produk.
Membaca file halaman8.html...
--> Ditemukan 80 calon produk.
Membaca file halaman9.html...
--> Ditemukan 16 calon produk.

================ SUKSES ================
Data berhasil disimpan ke laptop_tokopedia_offline2.csv
Total Data Unik: 220
                                         Nama_Produk     Harga
0  Laptop Lenovo Ideapad Slim 3 14 AMD Ryzen 5 56...   8249000
1    LENOVO LEGION GLASSES GEN 2 MICRO OLED VR GLASS   6249000
2  Laptop ASUS Vivobook S14 M3407HA AMD Ryzen 7 2...  12799000
3  Laptop ASUS Vivobook S14 S3407CA Intel Core Ul...  12099000
4  Laptop ASUS Vivobook S14 S3407QA Snapdragon X1...   9539000
